In [1]:
import os

In [2]:
%pwd

'c:\\Users\\DELL\\Desktop\\Python DSA\\MLOps\\End_to_End_ML_Pipeline\\research'

In [3]:
os.chdir("../")
%pwd

'c:\\Users\\DELL\\Desktop\\Python DSA\\MLOps\\End_to_End_ML_Pipeline'

In [8]:
#Defining Data IUngestion entity as per workflow in readme
from dataclasses import dataclass
from pathlib import Path

#We are defining an ingestion class here
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
#Importing paths for schema/config/params yaml files from constants __inin__.py
# Here * means import everyting from this folder
from ml_wine_project.constants import *
from ml_wine_project.utils.common import read_yaml, create_directories

In [7]:
CONFIG_FILE_PATH

WindowsPath('config/config.yaml')

In [9]:
#Importing all yaml files in this class and creating the root directory named artifasct where we will ingest our data
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [10]:
import os
import urllib.request as request
import zipfile
from ml_wine_project import logger
from ml_wine_project.utils.common import get_size

In [11]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        """
        This finction will download the data file from URL and save it in the artifact folder
        """
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [12]:
#This is the ingestion pipeline where all the ingestion code will be executed
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-11-21 23:27:31,354: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-11-21 23:27:31,390: INFO: common: yaml file: params.yaml loaded successfully]
[2025-11-21 23:27:31,452: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-11-21 23:27:31,456: INFO: common: created directory at: artifacts]
[2025-11-21 23:27:31,465: INFO: common: created directory at: artifacts/data_ingestion]
[2025-11-21 23:27:32,842: INFO: 2801179986: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: F965:24FFFA:C296:145A8:6920A80C
Accept-Ranges: bytes
Date: Fri, 21 No